# 01. Data preprocessing for behavioral market segmentation

### Before the beginning, I strongly recommend you to read the documents describing the data. They are located here: "project_root/data/raw_data/"

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

For the purposes of extracting behavioral data, I need 1 sheet - "DurData_June 24, 2011", from the dataset.

In [2]:
transactions_df = pd.read_excel('../../data/raw_data/data.xlsx', sheet_name = "DurData_June 24, 2011")

In [3]:
initial_data = transactions_df.copy()

Some information about the data:

In [4]:
transactions_df.isna().sum()

Household_ID                        0
Transaction_NBR                     0
Transaction_Total                   0
Transaction_Date                    0
Transaction_Location                0
Online_Transaction                  0
ORIGINAL_TICKET_NBR                 0
Transaction_type                    0
PRODUCT_ID                      41163
Category_Description              517
Sub_Category_NBR                    0
Sub_Category_Description         3847
Transaction_Type_Description       36
Quantity                            0
UNIT_PRICE                          0
EXTENDED_PRICE                      0
RETURN_IND                      13411
Return_Location_If Any              0
Age_H.Head                      20971
CHILDERN_PRESENCE               91777
Income                          20455
GENDER_H.Head                       0
Gender_Individual               14575
MALE_CHID_AGE_0-2                   0
MALE_CHID_AGE_3-5                   0
MALE_CHID_AGE_6-10                  0
MALE_CHID_AG

In [5]:
transactions_df.head()

,Household_ID,Transaction_NBR,Transaction_Total,Transaction_Date,Transaction_Location,Online_Transaction,ORIGINAL_TICKET_NBR,Transaction_type,PRODUCT_ID,Category_Description,...,FEMALE_CHID_AGE_0-2,FEMALE_CHID_AGE_3-5,FEMALE_CHID_AGE_6-10,FEMALE_CHID_AGE_11-15,FEMALE_CHID_AGE_16-17,UNKNOWN_CHID_AGE_0-2,UNKNOWN_CHID_AGE_3-5,UNKNOWN_CHID_AGE_6-10,UNKNOWN_CHID_AGE_11-15,UNKNOWN_CHID_AGE_16-17
0,100003544,1,1,06JUN2003:00:00:00,537,0,53703232083,1,879852.0,VIDEO HDWR,...,0,0,0,0,0,0,0,0,0,0
1,100012312,1,2,09JAN2001:00:00:00,841,0,84102381410,1,580994.0,TELEVISION,...,0,0,0,0,0,0,0,0,0,0
2,100012312,2,2,09JAN2001:00:00:00,841,0,84102381410,1,720025.0,P*S*T,...,0,0,0,0,0,0,0,0,0,0
3,100016237,1,1,22JAN2001:00:00:00,949,0,94904161842,1,575635.0,P*S*T,...,0,0,0,0,0,0,0,0,0,0
4,100022945,1,11,30JAN2001:00:00:00,521,0,52101921224,1,545443.0,MOBILE,...,0,0,0,0,1,0,0,0,0,0


---

I leave only 'buy' transactions, which have not been returned.

In [6]:
transactions_df = transactions_df[transactions_df.Transaction_type == 1]

In [7]:
transactions_df = transactions_df[transactions_df.RETURN_IND == 'N']

Leave only relevant columns for analysis: "Household_ID" - to indentify the customer. "Category_Description", 'Quantity' and "EXTENDED_PRICE" - UNIT_PRICE * Quantity.

In [8]:
transactions_df = transactions_df[['Household_ID', 'Category_Description', 'Quantity', 'EXTENDED_PRICE']]

Leaving only transactions with defined category.

In [9]:
transactions_df = transactions_df[transactions_df.Category_Description.notna()]

To find how many items were bought and how much money were spent in each particular category I group the data, by Household_ID and Category_Description and use "sum" aggregate function:

In [10]:
transactions_df = transactions_df.groupby(['Household_ID','Category_Description'], as_index = False).sum()

Now each row in data frame, represents: Houshold_id, category, total number of items bought in this category,total amount of money spent in this category.

In [11]:
transactions_df.head()

,Household_ID,Category_Description,Quantity,EXTENDED_PRICE
0,100003544,VIDEO HDWR,1,99.97
1,100012312,P*S*T,1,19.99
2,100012312,TELEVISION,1,9.99
3,100016237,P*S*T,1,89.99
4,100022945,IMAGING,1,259.99


### For market segmantation we need a new data frame, where each row will have housholdId, number of items bought in each category and amount of money spent in each category. 

I will use *pivot_table* API once for a quantity, second time for expenditures. Then I will merge the result.

In [12]:
transactions_df_q = transactions_df.pivot_table(index = 'Household_ID', columns = 'Category_Description',\
                            values = 'Quantity', aggfunc = np.sum, fill_value = 0, dropna = False)\
                    .reset_index().rename_axis(None, axis=1)

In [13]:
transactions_df_q.set_index(keys = 'Household_ID', inplace = True)

In [14]:
transactions_df_q.columns = transactions_df_q.columns + '_QNT'

Below, you can see the result of the code above. That is exactly what I needed.

In [15]:
transactions_df_q.head()

,AUDIO_QNT,DVS_QNT,EXPRESS_QNT,GIFT CARDS_QNT,HOME INS_QNT,IMAGING_QNT,INTABGIBLE_QNT,MAJORS_QNT,MOBILE_QNT,MUSIC_QNT,OTHER_QNT,P*S*T_QNT,PC HDWR_QNT,TELEVISION_QNT,VIDEO HDWR_QNT,WIRELESS_QNT
Household_ID,,,,,,,,,,,,,,,,
100003544,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
100012312,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
100016237,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
100022945,0,0,0,0,0,1,0,0,2,0,0,5,1,0,0,0
100022976,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Now let's do it second time for expenditures.

In [16]:
transactions_df_e = transactions_df.pivot_table(index = 'Household_ID', columns = 'Category_Description',\
                            values = 'EXTENDED_PRICE', aggfunc = np.sum, fill_value = 0, dropna = False)\
                    .reset_index().rename_axis(None, axis=1)

In [17]:
transactions_df_e.set_index(keys = 'Household_ID', inplace = True)

In [18]:
transactions_df_e.columns = transactions_df_e.columns + '_EXP'

In [19]:
transactions_df_e.head()

,AUDIO_EXP,DVS_EXP,EXPRESS_EXP,GIFT CARDS_EXP,HOME INS_EXP,IMAGING_EXP,INTABGIBLE_EXP,MAJORS_EXP,MOBILE_EXP,MUSIC_EXP,OTHER_EXP,P*S*T_EXP,PC HDWR_EXP,TELEVISION_EXP,VIDEO HDWR_EXP,WIRELESS_EXP
Household_ID,,,,,,,,,,,,,,,,
100003544,0.00,0.0,0.0,0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,99.97,0.0
100012312,0.00,0.0,0.0,0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,19.99,0.00,9.99,0.00,0.0
100016237,0.00,0.0,0.0,0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,89.99,0.00,0.00,0.00,0.0
100022945,0.00,0.0,0.0,0,0.0,259.99,0.0,0.0,24.98,0.0,0.0,193.95,149.99,0.00,0.00,0.0
100022976,599.99,0.0,0.0,0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,249.99,0.0


Merging everything in one data frame.

In [20]:
behavioral_df = transactions_df_q.merge(transactions_df_e, left_index = True, right_index = True)

Check the consistency of data reshaping outcome and the initial data.

In [21]:
behavioral_df[behavioral_df['AUDIO_QNT'] == behavioral_df['AUDIO_QNT'].max()][['AUDIO_QNT', 'AUDIO_EXP']]

,AUDIO_QNT,AUDIO_EXP
Household_ID,,
106680329,361,2801.29


In [22]:
initial_data[(initial_data.Household_ID == 106680329) & \
       (initial_data.Category_Description == 'AUDIO') & (initial_data.RETURN_IND == 'N')\
        & (initial_data.Transaction_type == 1)].EXTENDED_PRICE.sum()

2801.29

In [23]:
initial_data[(initial_data.Household_ID == 106680329) & \
       (initial_data.Category_Description == 'AUDIO') & (initial_data.RETURN_IND == 'N')\
       & (initial_data.Transaction_type == 1)].Quantity.sum()

361

Our behavioral data set, was constructed, under several assumptions:
- we took into account only 'buy' transactions, which were not returned to the store later.
- category of a good, should be defined. i.e. no 'na' values.

In the resulting behavioral data, we report number of items, bought, not the number of transactions.

In [24]:
behavioral_df.to_csv("../../data/processed_data/behavioral_data.csv")